---
format:
  html:
    code-links:
      - href: lite/lab/index.html?path=finding-poles.ipynb
        icon: play
        text: Run code
---

# Finding pole positions

:::{.callout-tip}
## Task

We have been gifted a $K$-matrix parameterization for two resonances within one channel, because it is International $K$-Matrix Day. The parameter values are:

- Masses of final state particles: $m_a=0.1\,\mathrm{GeV}$ and $m_b=0.2\,\mathrm{GeV}$
- Bare masses of the resonances: $m_1=1.8\,\mathrm{GeV}$ and $m_2=1.1\,\mathrm{GeV}$
- Couplings: $g=0.5\,\mathrm{GeV}$ and $g=0.7\,\mathrm{GeV}$
:::

:::{.callout-caution}
## Goal

Find **pole positions** and the **residues** of the two resonances with the given parameter values.
:::

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from IPython.display import Math

## Phase space factors

We define the phase space factor $\rho^\mathrm{CM}(s)$ using the Chew–Mandelstam function $\Sigma(s)$ for $S$&nbsp;waves ($L=0$). For the discontinuity between the sheets, we use the 'standard' phase space factor $\rho(s)$.

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    phi: Any = -np.pi / 2
    _latex_repr_ = R"\rho\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2, phi = self.args

        term1 = sp.sqrt(sp.exp(sp.I * phi) * (sp.sqrt(s) - (m1 + m2)))
        term2 = sp.exp(-sp.I * phi / 2)
        term3 = sp.sqrt(sp.sqrt(s) + (m1 + m2))
        term4 = sp.sqrt(s - (m1 - m2) ** 2)

        return term1 * term2 * term3 * term4 / s


@unevaluated(real=False)
class PhaseSpaceFactorCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


args = sp.symbols("s m_a m_b")
exprs = [
    PhaseSpaceFactor(*args),
    PhaseSpaceFactorCM(*args),
    ChewMandelstam(*args),
    BreakupMomentum(*args),
]
Math(aslatex({expr: expr.doit(deep=False) for expr in exprs}))

Compare [PDG 2024, Figure 50.6](https://pdg.lbl.gov/2024/reviews/rpp2024-rev-resonances.pdf#page=15):

In [ ]:
rho_func = sp.lambdify(args, PhaseSpaceFactor(*args).doit())
rho_cm_func = sp.lambdify(args, PhaseSpaceFactorCM(*args).doit())
x = np.linspace(-0.0, 1.3, num=500)
epsilon = 1e-8j
threshold = dict(m_a=0.13, m_b=0.5)
z_rho = 1j * rho_func(x + epsilon, **threshold)
z_rho_cm = 1j * rho_cm_func(x + epsilon, **threshold)

fig, axes = plt.subplots(figsize=(9, 4), ncols=2, sharey=True)
ax1, ax2 = axes
ax1.plot(x, z_rho.real, c="black", label="Real part", ls="--")
ax1.plot(x, z_rho.imag, c="red", label="Imag part")
ax2.plot(x, z_rho_cm.real, c="black", label="Real part", ls="--")
ax2.plot(x, z_rho_cm.imag, c="red", label="Imag part")
ax1.set_title(R"$i\rho(s)$")
ax2.set_title(R"$i\rho^{CM}(s)$")
s_thr = sum(threshold.values()) ** 2
for ax in axes:
    ax.axhline(0, alpha=0.5, c="black", lw=0.3)
    ax.axvline(s_thr, c="black", lw=0.5)
    ax.legend(loc="lower right")
    ax.set_xlabel(R"$s+i\epsilon$ [GeV$^2$]")
    ax.set_ylim(-1, +1)
fig.tight_layout()
plt.show(fig)

## Formulate $T$-matrix